For this second sweep the following changes have been made:
 - Parameters have been reduced to the range around the optimal parameter found during sweep 1.
 - A new parameter called dataset is added that randomly chooses between strib and kolding datasets.
 - Some parameters have been defaulted such as optimizer and lookback.
     - This is done to improve the other parameters even more, as these parameters showed to be the best choice in run 1.
 - For the sake of experiment, I will add an option for zero additional layers to the number of layers parameter.
 - Seeds have been added.
 - Number of Epochs have been reduced.

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import os
from utils import processing
from utils import utils

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import LSTM, GRU, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, TimeDistributed, \
    BatchNormalization
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import wandb
from wandb.keras import WandbCallback


In [21]:
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
tf.random.set_seed(hash("by removing stochasticity") % 2**32 - 1)

In [22]:
plt.style.use('fivethirtyeight')
plt.rcParams["figure.figsize"] = (18,10)

In [23]:
wandb.login()

True

In [24]:
def load_data(config):
    # Load csv & parse dates to datetime index
    data = pd.read_csv(f'../data/processed/{config.dataset}_features.csv', index_col='Datetime', parse_dates=['Datetime'])
    # Select Features
    data = data[['Value', 'sunshine_mins', 'airtemp_c', 'daylength_hrs', 'wkdy_sin', 'wkdy_cos', 'wknd', 'mnth_sin', 'mnth_cos']]
    
    train, val, test = processing.create_datasets(data, split=split, 
                                                  steps=steps, lookback=config.lookback, 
                                                  horizon=horizon, batch_size=config.batch_size, 
                                                  scaler='standard')    
    return train, val, test
    
    
def build_model(config):
        
    model = Sequential()

    model.add(Input(shape=(config.lookback, config.num_features)))

    # CNN Block
    model.add(Conv1D(filters=config.cnn_layer_size_1, kernel_size=3, activation=config.activation_cnn))
    model.add(MaxPooling1D(pool_size=2))    

    model.add(Conv1D(filters=config.cnn_layer_size_2, kernel_size=3, activation=config.activation_cnn))
    model.add(MaxPooling1D(pool_size=2))
    
    model.add(Conv1D(filters=config.cnn_layer_size_3, kernel_size=3, activation=config.activation_cnn))
    model.add(MaxPooling1D(pool_size=2))
    
    # RNN Block

    model.add(GRU(config.gru_layer_size_1, return_sequences=True, activation=config.activation_gru))
    model.add(Dropout(config.dropout))
    
    model.add(GRU(config.gru_layer_size_2, return_sequences=True, activation=config.activation_gru))
    model.add(Dropout(config.dropout))
    
    model.add(GRU(config.gru_layer_size_3, return_sequences=False, activation=config.activation_gru))
    model.add(Dropout(config.dropout))

    model.add(Dense(1))
    
    opt = config.optimizer
    if opt == 'sgd':
        opt = SGD(learning_rate=config.learning_rate, momentum=config.momentum)
    elif opt == 'rmsprop':
        opt = RMSprop(learning_rate=config.learning_rate)
    else:
        opt = Adam(learning_rate=config.learning_rate)
        
    model.compile(optimizer=opt, loss='mse', metrics=['mae', 'mape'])
    
    return model

def log_defaults():
    # Default values from the first sweep
    # The following values resulted in the strongest model
    wandb_config = {
        'num_features' : 9,
        'epochs' : 50,
        'batch_size' : 227,
        'num_cnn_layers' : 2, # Number of additional layers
        'num_gru_layers' : 2, # Number of additional layers
        'optimizer' : 'sgd',
        'dropout' : 0.015,
        'lookback' : 718,
        'activation_cnn' : 'relu',
        'activation_gru' : 'tanh',
        'cnn_layer_size_1' : 70,
        'cnn_layer_size_2' : 106,
        'cnn_layer_size_3' : 106,
        'gru_layer_size_1' : 51,
        'gru_layer_size_2' : 194,
        'gru_layer_size_3' : 194,
        'learning_rate' : 0.024,
        'momentum' : 0.9,
        'dataset' : 'kolding'
    }
    return wandb_config
    

def run_tuner():
    
    wandb.init(config=log_defaults(), group='cnnrnn-sweep-3', project='thesis')
    
    model = build_model(config=wandb.config)
    
    train, val, _ = load_data(config=wandb.config)
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.6, min_lr=0.001, patience=5)
    
    callbacks = [WandbCallback(), reduce_lr]
    
    model.fit(
        train,
        epochs=wandb.config.epochs,
        validation_data=val,
        callbacks=callbacks
    )

In [25]:
# Sweep Config
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'early_terminate': {
        'type': 'hyperband',
        'min_iter': 5
    },
    'parameters': {
        'num_features' : {
            'value' : 9
        },
        'batch_size': {
            'distribution': 'q_log_uniform',
            'q': 1,
            'min': math.log(32),
            'max': math.log(256)
        },
        'lookback': {
            'value': 24*31
        },
        'optimizer': {
            'value': 'sgd'
        },
        'dropout': {
            'distribution': 'uniform',
            'min': 0.01,
            'max': 0.5
        },
        'epochs': {
            'value': 25
        },
        'activation_gru': {
            'value': 'tanh'
        },
        'activation_cnn': {
            'value': 'relu'
        },
        'cnn_layer_size_1': {
            'distribution': 'q_log_uniform',
            'q': 1,
            'min': math.log(32),
            'max': math.log(256)
        },
        'cnn_layer_size_2': {
            'distribution': 'q_log_uniform',
            'q': 1,
            'min': math.log(32),
            'max': math.log(256)
        },
        'cnn_layer_size_3': {
            'distribution': 'q_log_uniform',
            'q': 1,
            'min': math.log(32),
            'max': math.log(256)
        },
        'gru_layer_size_1': {
            'distribution': 'q_log_uniform',
            'q': 1,
            'min': math.log(32),
            'max': math.log(256)
        },
        'gru_layer_size_2': {
            'distribution': 'q_log_uniform',
            'q': 1,
            'min': math.log(32),
            'max': math.log(256)
        },
        'gru_layer_size_3': {
            'distribution': 'q_log_uniform',
            'q': 1,
            'min': math.log(32),
            'max': math.log(256)
        },
        'learning_rate': {
            'distribution': 'uniform',
            'min': 0.001,
            'max': 0.04
        },
        'momentum': {
            'distribution': 'uniform',
            'min': 0.5,
            'max': 1
        },
        'num_cnn_layers': {
            'value': 2
        },
        'num_gru_layers': {
            'value': 2
        },
        'dataset': {
            'values': ['kolding', 'strib']
        }
    }
}

In [26]:
# Additional Parameters
split = 0.80 # split percentage for training data
steps = 1 # timesteps: 1 hour
horizon = 1 # the target hour in the future we want to predict 1 hour ahead


In [27]:
sweep_id = wandb.sweep(sweep_config, project='thesis')

Create sweep with ID: f6tc66bl
Sweep URL: https://wandb.ai/nbvanting/thesis/sweeps/f6tc66bl


In [28]:
wandb.agent(sweep_id, function=run_tuner, count=10)

wandb: Agent Starting Run: vzdog0go with config:
wandb: 	activation_cnn: relu
wandb: 	activation_gru: tanh
wandb: 	batch_size: 180
wandb: 	cnn_layer_size_1: 92
wandb: 	cnn_layer_size_2: 94
wandb: 	cnn_layer_size_3: 89
wandb: 	dataset: kolding
wandb: 	dropout: 0.3128109462373897
wandb: 	epochs: 25
wandb: 	gru_layer_size_1: 33
wandb: 	gru_layer_size_2: 85
wandb: 	gru_layer_size_3: 63
wandb: 	learning_rate: 0.009599069072943795
wandb: 	lookback: 744
wandb: 	momentum: 0.9184209591699685
wandb: 	num_cnn_layers: 2
wandb: 	num_features: 9
wandb: 	num_gru_layers: 2
wandb: 	optimizer: sgd
wandb: WARNING Ignored wandb.init() arg project when running a sweep
c:\users\nicol\envs\thesis\lib\site-packages\IPython\html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "
wandb: wandb version 0.10.29 is

Epoch 1/25
81/81 [==============================] - 53s 579ms/step - loss: 0.7962 - mae: 0.6994 - mape: 362.8096 - val_loss: 1.3632 - val_mae: 0.8970 - val_mape: 210.3083
Epoch 2/25
81/81 [==============================] - 45s 558ms/step - loss: 0.7166 - mae: 0.6660 - mape: 312.5047 - val_loss: 1.3332 - val_mae: 0.9141 - val_mape: 242.1300
Epoch 3/25
81/81 [==============================] - 45s 556ms/step - loss: 0.6809 - mae: 0.6493 - mape: 343.6742 - val_loss: 1.2124 - val_mae: 0.8796 - val_mape: 227.2636
Epoch 4/25
81/81 [==============================] - 46s 569ms/step - loss: 0.6297 - mae: 0.6328 - mape: 628.4665 - val_loss: 0.6221 - val_mae: 0.6143 - val_mape: 190.1386
Epoch 5/25
81/81 [==============================] - 45s 557ms/step - loss: 0.4328 - mae: 0.5148 - mape: 484.3412 - val_loss: 0.4155 - val_mae: 0.5074 - val_mape: 177.8502
Epoch 6/25
81/81 [==============================] - 45s 555ms/step - loss: 0.3364 - mae: 0.4514 - mape: 530.2902 - val_loss: 0.2913 - val_mae: 0.

epoch,24
loss,0.11301
mae,0.25883
mape,282.91248
val_loss,0.1174
val_mae,0.27669
val_mape,106.74454
_runtime,1176
_timestamp,1620139681
_step,24
best_val_loss,0.1097


epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
loss,██▇▆▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
mae,███▇▅▄▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
mape,▄▃▃▆▇▆█▆▆▅▆▃▃▃▄▃▃▃▃▁▅▂▂▃▂
val_loss,██▇▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mae,███▅▄▃▃▂▂▂▂▂▂▁▁▁▁▂▁▁▁▁▁▁▁
val_mape,▆█▇▆▅▄▄▃▃▂▂▂▂▂▂▁▂▂▁▁▁▁▁▂▂
_runtime,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▆▆▆▆▇▇▇██
_timestamp,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▆▆▆▆▇▇▇██
_step,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██


wandb: Agent Starting Run: wbxmnktc with config:
wandb: 	activation_cnn: relu
wandb: 	activation_gru: tanh
wandb: 	batch_size: 107
wandb: 	cnn_layer_size_1: 89
wandb: 	cnn_layer_size_2: 52
wandb: 	cnn_layer_size_3: 45
wandb: 	dataset: kolding
wandb: 	dropout: 0.2722160418051974
wandb: 	epochs: 25
wandb: 	gru_layer_size_1: 60
wandb: 	gru_layer_size_2: 75
wandb: 	gru_layer_size_3: 176
wandb: 	learning_rate: 0.014346654394248926
wandb: 	lookback: 744
wandb: 	momentum: 0.6199037047603387
wandb: 	num_cnn_layers: 2
wandb: 	num_features: 9
wandb: 	num_gru_layers: 2
wandb: 	optimizer: sgd
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: wandb version 0.10.29 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


...

epoch,▁▂▂▃▄▄▅▅▆▇▇█
loss,█▇▇▇▇▇▆▆▄▂▂▁
mae,███▇▇▇▇▆▄▃▂▁
mape,▂▂▂▁▁▂▂▃▆▆▇█
val_loss,████▇▇▇▅▃▂▂▁
val_mae,▇▇██▇█▇▆▃▂▂▁
val_mape,▄▆██▅▇▄▆▁▂▄▁
_runtime,▁▂▂▃▄▄▅▅▆▇▇█
_timestamp,▁▂▂▃▄▄▅▅▆▇▇█
_step,▁▂▂▃▄▄▅▅▆▇▇█


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 7bxsbwo7 with config:
wandb: 	activation_cnn: relu
wandb: 	activation_gru: tanh
wandb: 	batch_size: 147
wandb: 	cnn_layer_size_1: 32
wandb: 	cnn_layer_size_2: 78
wandb: 	cnn_layer_size_3: 38
wandb: 	dataset: kolding
wandb: 	dropout: 0.08791089961754245
wandb: 	epochs: 25
wandb: 	gru_layer_size_1: 133
wandb: 	gru_layer_size_2: 48
wandb: 	gru_layer_size_3: 108
wandb: 	learning_rate: 0.014745383918515442
wandb: 	lookback: 744
wandb: 	momentum: 0.7736504000202488
wandb: 	num_cnn_layers: 2
wandb: 	num_features: 9
wandb: 	num_gru_layers: 2
wandb: 	optimizer: sgd
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: wandb version 0.10.29 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/25
100/100 [==============================] - 70s 634ms/step - loss: 0.7952 - mae: 0.6919 - mape: 303.0058 - val_loss: 1.3751 - val_mae: 0.9118 - val_mape: 243.5748
Epoch 2/25
100/100 [==============================] - 59s 587ms/step - loss: 0.7122 - mae: 0.6647 - mape: 457.5337 - val_loss: 1.3531 - val_mae: 0.9172 - val_mape: 254.8871
Epoch 3/25
100/100 [==============================] - 57s 571ms/step - loss: 0.6855 - mae: 0.6538 - mape: 390.8345 - val_loss: 1.4204 - val_mae: 0.8734 - val_mape: 136.2679
Epoch 4/25
100/100 [==============================] - 56s 557ms/step - loss: 0.6786 - mae: 0.6521 - mape: 391.6583 - val_loss: 1.2122 - val_mae: 0.8811 - val_mape: 219.8545
Epoch 5/25
100/100 [==============================] - 55s 554ms/step - loss: 0.6193 - mae: 0.6277 - mape: 417.9128 - val_loss: 0.7725 - val_mae: 0.6849 - val_mape: 172.0443
Epoch 6/25
100/100 [==============================] - 54s 543ms/step - loss: 0.4524 - mae: 0.5318 - mape: 684.5988 - val_loss: 0.5005 -

epoch,6
loss,0.32084
mae,0.44195
mape,713.81537
val_loss,1.00268
val_mae,0.83251
val_mape,278.09988
_runtime,411
_timestamp,1620154097
_step,6
best_val_loss,0.50048


epoch,▁▂▃▅▆▇█
loss,█▇▇▇▅▃▁
mae,███▇▆▃▁
mape,▁▂▁▂▂▄█
val_loss,█▇█▆▃▁▅
val_mae,██▇▇▄▁▆
val_mape,▆▇▁▅▃▂█
_runtime,▁▂▃▅▆▇█
_timestamp,▁▂▃▅▆▇█
_step,▁▂▃▅▆▇█
